# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#import current weather data from csv
clean_weather_data = pd.read_csv('../Output/clean_weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_weather_data[["Latitude", "Longitude"]]
humidity = clean_weather_data['Humidity (%)']

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Narrowing Data to My Ideal Weather Conditions
My ideal weather conditions include:
- Anywhere between 75 and 80 degrees.
- A pleasent breeze (5 - 24 mph)
- Some clouds in the sky to admire (20% to 50%)

In [5]:
# narrowing the data to my ideal weather conditions
ideal_weather_data = clean_weather_data.loc[(clean_weather_data['Temperature (F)'] >= 75) & (clean_weather_data['Temperature (F)'] < 80),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Wind Speed (mph)'] >= 5) & (ideal_weather_data['Wind Speed (mph)'] < 24),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Cloudiness (%)'] >= 20) & (ideal_weather_data['Cloudiness (%)'] < 50),:]

ideal_weather_data = ideal_weather_data.set_index('City')
ideal_weather_data

,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,
hami,42.8000,93.4500,78.91,78.91,25,20,8.10
salinopolis,-0.6136,-47.3561,78.98,78.98,83,26,13.15
bathsheba,13.2167,-59.5167,79.41,79.41,89,20,8.05
humberto de campos,-2.5983,-43.4611,78.04,78.04,79,22,6.15
puerto ayora,-0.7393,-90.3518,75.16,75.16,75,35,11.77
sao filipe,14.8961,-24.4956,78.46,78.46,86,48,8.08
maneadero,31.7167,-116.5667,75.09,77.18,75,23,5.01
indramayu,-6.3264,108.3200,78.12,78.12,83,41,8.23
samarai,-10.6167,150.6667,78.53,78.53,86,27,16.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(columns = ['City','Hotel Name','Location', 'Rating'])
hotel_df = hotel_df.set_index('City')
hotel_df

,Hotel Name,Location,Rating
City,,,


In [7]:
#test_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=29.4241,-98.4936&radius=5000&type=hotel&key={g_key}'
#test_response = requests.get(test_url).json()
#test_response['results'][1]

In [8]:
# find the closest restaurant of each type to coordinates

search_radius= 5000
search_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
query_perams = f'&radius={search_radius}&type={search_type}&key={g_key}'

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_data.iterrows():
  
    lat = row['Latitude']
    lng = row['Longitude']
        
    # get location type from df
    location = f'location={lat},{lng}'
    
    # assemble url and make API request
    print(f"Searching for Hotel in {index}.")
    query_url = base_url + location + query_perams
    response = requests.get(query_url).json()   
    
    if response['status'] == 'OK':
    
        # results code stem
        results = response['results'][1]
     
        try: 
            hotel_df.loc[index, 'Hotel Name'] = results['name']
            hotel_df.loc[index, 'Location'] = (results['plus_code']['compound_code'])[8:]
            hotel_df.loc[index, 'Rating'] = results['rating']
            hotel = results['name']
            print(f'Hotel: {hotel}')
        
        except (KeyError, IndexError):
            print("No Hotels Found Nearby... Skipping.")
    else:
        print("No Hotels Found Nearby... Skipping.")

    print("------------")

Searching for Hotel in hami.
No Hotels Found Nearby... Skipping.
------------
Searching for Hotel in salinopolis.
Hotel: Salinas Park Resort Maçarico AP101
------------
Searching for Hotel in bathsheba.
No Hotels Found Nearby... Skipping.
------------
Searching for Hotel in humberto de campos.
Hotel: Banco do Brasil
------------
Searching for Hotel in puerto ayora.
Hotel: Finch Bay Galapagos Hotel
------------
Searching for Hotel in sao filipe.
Hotel: Hotel Xaguate
------------
Searching for Hotel in maneadero.
No Hotels Found Nearby... Skipping.
------------
Searching for Hotel in indramayu.
Hotel: SD Negeri Pabean Udik II
------------
Searching for Hotel in samarai.
Hotel: Nuli Sapi
------------
Searching for Hotel in sinnamary.
No Hotels Found Nearby... Skipping.
------------


In [9]:
hotel_df = pd.merge(hotel_df,ideal_weather_data, on = ['City'], how='left')
hotel_df = hotel_df.dropna(how='any')
hotel_df

,Hotel Name,Location,Rating,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,,,,
salinopolis,Salinas Park Resort Maçarico AP101,"Salinópolis - State of Pará, Brazil",3,-0.6136,-47.3561,78.98,78.98,83,26,13.15
humberto de campos,Banco do Brasil,"Humberto de Campos, MA, Brazil",3.4,-2.5983,-43.4611,78.04,78.04,79,22,6.15
puerto ayora,Finch Bay Galapagos Hotel,"Puerto Ayora, Ecuador",4.7,-0.7393,-90.3518,75.16,75.16,75,35,11.77
sao filipe,Hotel Xaguate,"Sao Filipe, Cape Verde",3.4,14.8961,-24.4956,78.46,78.46,86,48,8.08
indramayu,SD Negeri Pabean Udik II,"Pabeanudik, Indramayu Regency, West Java, Indo...",5,-6.3264,108.3200,78.12,78.12,83,41,8.23
samarai,Nuli Sapi,"Kasabanalua, Papua New Guinea",5,-10.6167,150.6667,78.53,78.53,86,27,16.02


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<c><dl>
<dt>Location:</dt><dd>{Location}</dd>
<dt>Hotel:</dt><dd>{Hotel Name}</dd>
<dd>Rated {Rating} out of 5</dd>
<dt>
</c></dl>
"""

In [11]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hover = hotel_df['Location']

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hover_text=hover, label='', info_box_content=hotel_info, display_info_box=None)
fig1.add_layer(marker_layer)

## The only thing left is to book my flight!
Below are all the locations lucky enough to be currently experiencing my ideal weather conditions. Click on each pin to view a hotel in the area. 

In [13]:
# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))